# Example: Ingest Real-Time Stock Data to iguazio NoSQL and TSDB
the following example function ingest real-time stock information from an internet service (World Trading Data) into iguazio platform.<br>
everytime the data is updated it updates a NoSQL table with the recent metadata and updates the TSDB with the new metrics (price and volume)

The same code can run inside a nuclio (serverless) function and be automatically triggered on a predefined schedule (cron) or through HTTP requests

### Initialization (not requiered in nuclio)
need to fill the following environment variables with real credentials.<br>
obtain your free API token from [World Trading Data](https://www.worldtradingdata.com) <br>
> note: in nuclio environment variables and build dependencies are specified in the configuration tab

In [ ]:
# nuclio: ignore
from nuclio import Context, Event

%env API_TOKEN <WorldTradingData API Token>
%env V3IO_PASSWORD <V3IO-Password>
%env V3IO_USER <V3IO-Username>
%env V3IO_ADDRESS <address of V3IO API end point>

context = Context()

### function implementation

In [ ]:
import json
import v3io
import requests
import os
import pandas as pd
import datetime

stocks = os.getenv('STOCK_LIST','GOOG,MSFT,AMZN,IBM,ORCL')
token = os.getenv('API_TOKEN') 
url = 'https://www.worldtradingdata.com/api/v1/stock?symbol={0}&api_token={1}'.format(stocks,token)
v3 = v3io.v3io(os.getenv('V3IO_ADDRESS'),os.getenv('V3IO_USER'),os.getenv('V3IO_PASSWORD'), 'bigdata')

# v3io update expression template 
expr_template = "symbol='{symbol}';name='{name}';currency='{currency}';exchange='{stock_exchange_short}';" + \
    "timezone='{timezone}';price={price};volume={volume};last_trade='{last_trade_time}'"

last_trade_times = {}

# nuclio habdler fuction 
def handler(context, event):
    
    # reading latest stock information 
    r = requests.get(url)
    
    for stock in r.json()['data']:
        
        symbol = stock['symbol']
        last = last_trade_times.get(symbol)
        date = datetime.datetime.strptime(stock['last_trade_time'], '%Y-%m-%d %H:%M:%S')
        
        # update the stocks table and TSDB metrics in case of new data 
        if not last or date > last:
            # update NoSQL table with stock data
            expr = expr_template.format(**stock)
            context.logger.debug_with('update expression', symbol=symbol, expr=expr)
            v3.updateitem('stocks',symbol, expr, '')
        
            # update time-series DB with price and volume metrics (use pandas dataframe with a single row, indexed by date)
            df = pd.DataFrame([[float(stock['price']), float(stock['volume'])]], columns=["price", "volume"], index=[date])
            write_tsdb('stock_metrics', df)
            
            last_trade_times[symbol] = date 
        
    return r.json()

### function invocation
the following section simulates nuclio function invocation

In [41]:
event = Event(body='')
handler(context, event)

{'symbols_requested': 5,
 'symbols_returned': 5,
 'data': [{'symbol': 'AMZN',
   'name': 'Amazon.com, Inc.',
   'currency': 'USD',
   'price': '1719.36',
   'price_open': '1724.00',
   'day_high': '1755.40',
   'day_low': '1685.10',
   '52_week_high': '2050.50',
   '52_week_low': '962.50',
   'day_change': '-35.89',
   'change_pct': '-2.04',
   'close_yesterday': '1755.25',
   'market_cap': '838602683574',
   'volume': '13836440',
   'shares': '487741000',
   'stock_exchange_long': 'NASDAQ Stock Exchange',
   'stock_exchange_short': 'NASDAQ',
   'timezone': 'EDT',
   'timezone_name': 'America/New_York',
   'gmt_offset': '-14400',
   'last_trade_time': '2018-10-11 16:00:02'},
  {'symbol': 'GOOG',
   'name': 'Alphabet Inc Class C',
   'currency': 'USD',
   'price': '1079.32',
   'price_open': '1072.94',
   'day_high': '1106.40',
   'day_low': '1068.27',
   '52_week_high': '1273.89',
   '52_week_low': '960.52',
   'day_change': '-1.90',
   'change_pct': '-0.18',
   'close_yesterday': '108